In [1]:
import torch 
from torch import nn 
from torch.optim import Adam
from torch.utils.data import DataLoader
from models import UNet, UNet16

import torch
import numpy as np
import cv2 
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
from albumentations.pytorch.transforms import img_to_tensor 
from torchvision.io import read_image
from argparse import ArgumentParser
import sys
from albumentations import (HorizontalFlip, VerticalFlip, Normalize, Compose, RandomCrop, RandomBrightnessContrast, PadIfNeeded, GaussNoise, OpticalDistortion, 
                             MotionBlur, HueSaturationValue)
from pathlib import Path
import progressbar
import random
import json
import datetime

In [2]:
import torch 
from torch import nn 
from torch.optim import Adam
from torch.utils.data import DataLoader
from models import UNet, UNet16

import torch
import numpy as np
import cv2 
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2
from albumentations.pytorch.transforms import img_to_tensor 
from torchvision.io import read_image
from argparse import ArgumentParser
import sys
from albumentations import (HorizontalFlip, VerticalFlip, Normalize, Compose, RandomCrop, RandomBrightnessContrast, PadIfNeeded, GaussNoise, OpticalDistortion, 
                             MotionBlur, HueSaturationValue)
from pathlib import Path
import progressbar
import random
import json
import datetime


def four_fold_cross_val(factor):

    val_dict  = {0: [1,8], 1: [5,6], 2: [3,7], 3: [2,4]}
    current_working_directory = Path.cwd()
    data_directory = current_working_directory/'data'
    data_location = data_directory/ 'cropped_images'

    print(data_location)

    validation_images =[]


    training_images = []

    for x in range(1,9):

        if x in val_dict[factor]:
            print(list((data_location / ('instrument_dataset_' + str(x))/'training_image_data').glob('*')))
            validation_images.extend(list((data_location / ('instrument_dataset_' + str(x))/'training_image_data').glob('*')))

        else:
            training_images.extend(list((data_location / ('instrument_dataset_' + str(x))/'training_image_data').glob('*')))
            print((data_location / ('instrument_dataset_' + str(x))/'training_image_data').glob('*'))
   
    return training_images, validation_images

b,r= four_fold_cross_val(1)

print(b)


/home/vivek/ai_in_surgery/project_code/data/cropped_images
<generator object Path.glob at 0x7f8051c32260>
<generator object Path.glob at 0x7f8051c32260>
<generator object Path.glob at 0x7f8051c32260>
<generator object Path.glob at 0x7f8051c32260>
[PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame131.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame095.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame074.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame119.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame079.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_5/training_image_data/frame2

In [18]:
print(four_fold_cross_val(0))

/home/vivek/ai_in_surgery/project_code/data/cropped_images
[PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame131.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame095.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame074.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame119.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame079.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame211.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/training_image_data/frame023.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/d

In [7]:

current_working_directory = Path.cwd()
data_directory = current_working_directory/'data'
data_location = data_directory/ 'cropped_images'

print(current_working_directory)
print(data_directory)

/home/vivek/ai_in_surgery/project_code
/home/vivek/ai_in_surgery/project_code/data


In [9]:
import cv2
import matplotlib.pyplot as plt

import numpy as np
image_path = "/home/vivek/ai_in_surgery/project_code/data/cropped_images/instrument_dataset_1/binary_masks/frame000.png"

image = cv2.imread(image_path,0)



unique_labels, label_counts = np.unique(image, return_counts=True)

print(unique_labels)



[  0 255]


In [ ]:
def process_image(file_path, base_dir, label_folders, coords):
    """Process an individual image file."""
    x_start, y_start, new_width, new_height = coords
    image = cv2.imread(str(file_path))
    cropped_image = image[y_start: y_start + new_height, x_start: x_start + new_width]

    cv2.imwrite(str(base_dir / 'training_image_data' / (file_path.stem + '.png')), cropped_image)

    # Initialize truth arrays
    truth_binary = np.zeros((new_height, new_width), dtype=np.uint8)
    truth_parts = np.zeros((new_height, new_width), dtype=np.uint8)
    truth_instruments = np.zeros((new_height, new_width), dtype=np.uint8)

    # Process each label folder
    for label_folder in label_folders:
        mask = cv2.imread(str(label_folder / file_path.name), 0)

        
        cropped_mask = mask[y_start: y_start + new_height, x_start: x_start + new_width]

        # Update truth arrays based on conditions
        # These conditions will need to be adapted to your specific case
        if 'Bipolar_Forceps' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 1
        elif 'Prograsp_Forceps' in str(label_folder):
                truth_instruments[cropped_mask > 0] = 2
        elif 'Large_Needle_Driver' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 3
        elif 'Vessel_Sealer' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 4
        elif 'Grasping_Retractor' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 5
        elif 'Monopolar_Curved_Scissors' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 6
        elif 'Other' in str(label_folder):
            truth_instruments[cropped_mask > 0] = 7
        
        if 'Other' not in str(label_folder):
            truth_binary+=cropped_mask

            truth_parts[cropped_mask == 10] = 1
            truth_parts[cropped_mask == 20] = 2
            truth_parts[cropped_mask == 30] = 3

    truth_binary[truth_binary > 0] = 255
    truth_binary = truth_binary.astype(np.uint8)
    truth_parts = truth_parts.astype(np.uint8) * 85
    truth_instruments = truth_instruments.astype(np.uint8) * 32

    # Save the masks
    cv2.imwrite(str(base_dir / 'binary_masks' / (file_path.stem + '.png')), truth_binary)
    cv2.imwrite(str(base_dir / 'parts_masks' / (file_path.stem + '.png')), truth_parts)
    cv2.imwrite(str(base_dir / 'instruments_masks' / (file_path.stem + '.png')), truth_instruments)

In [1]:
import os 
os.cpu_count()

8

In [22]:
from pathlib import Path

def prepare_test_images(test_images):
     image_list =[]
     for x in range(1,11):
          image_list.extend(list((test_images / ('instrument_dataset_' + str(x))/'left_frames').glob('*')))
     
     return image_list

current_working_directory = Path.cwd()

complete = current_working_directory/"data"/"test_set"

print(complete)
hello = prepare_test_images(complete)

    print(hello)

/home/vivek/ai_in_surgery/project_code/data/test_set
[PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame287.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame269.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame297.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame294.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame252.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame256.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame259.png'), PosixPath('/home/vivek/ai_in_surgery/project_code/data/test_set/instrument_dataset_1/left_frames/frame292.png'), PosixPath('/home/vivek/ai_in_surgery/proje

In [28]:
import numpy as np  
list_of_lists = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

print(np.sum(list_of_lists))

print(np.mean(list_of_lists))

45
5.0
